In [1]:
import pandas as pd
import os 
import re
import numpy as np

In [2]:
def load_seqkit_report(fpath):
    """A function to load a seqkit summary """
    pdf = pd.read_csv(fpath, sep=r"\s+")
    
    columns = [
        'num_seqs',
        'sum_len',
        'min_len',
        'avg_len',
        'max_len',
        'Q1',
        'Q2',
        'Q3',
        'N50',
    ] 
    
    for c in columns:
        pdf[c] = pdf[c].astype(str).str.replace(',', '').astype(float)
        
    pdf['Gb'] = pdf['sum_len'] / 1e+9
    columns = [
        'file',
        'num_seqs',
        'Gb',
        'avg_len',
        'N50',
    ]

    return pdf[columns]

raw_path = "/scratch/indikar_root/indikar1/shared_data/adapter_clean_up/reports/seqkit.report.txt"
clean_path = "/scratch/indikar_root/indikar1/shared_data/adapter_clean_up/reports/seqkit.porechop.report.txt"

raw = load_seqkit_report(raw_path)
clean = load_seqkit_report(clean_path)

df = pd.merge(raw, clean, 
              how='left', 
              left_on='file',
              right_on='file',
              suffixes=('_raw', '_trimmed'),
              )

df['cell'] = df['file'].str[2:5]
df['sample'] = df['file'].str[:2]

outpath = "~/temp/scpore_c_adapter_trimming_summary.csv" 
df.to_csv(outpath, index=False)
df.head()

,file,num_seqs_raw,Gb_raw,avg_len_raw,N50_raw,num_seqs_trimmed,Gb_trimmed,avg_len_trimmed,N50_trimmed,cell,sample
0,o4b01.raw.fastq,272951.0,0.158564,580.9,661.0,16284.0,0.000593,36.4,69.0,b01,o4
1,o4b02.raw.fastq,59988.0,0.028069,467.9,511.0,3314.0,0.000282,85.2,583.0,b02,o4
2,o4b03.raw.fastq,93482.0,0.143062,1530.4,1922.0,785.0,0.000052,66.6,1106.0,b03,o4
3,o4b04.raw.fastq,119114.0,0.072144,605.7,665.0,22499.0,0.018929,841.3,1502.0,b04,o4
4,o4b05.raw.fastq,212321.0,0.101544,478.3,516.0,20530.0,0.002451,119.4,639.0,b05,o4


In [3]:
df.columns

Index(['file', 'num_seqs_raw', 'Gb_raw', 'avg_len_raw', 'N50_raw',
       'num_seqs_trimmed', 'Gb_trimmed', 'avg_len_trimmed', 'N50_trimmed',
       'cell', 'sample'],
      dtype='object')

In [4]:

gx = df.groupby('sample').agg(
       n_cells = ('file', 'count'),
       raw_reads = ('num_seqs_raw', 'sum'),
       trimmed_reads = ('num_seqs_trimmed', 'sum'),
       raw_Gb = ('Gb_raw', 'sum'),
       trimmed_Gb = ('Gb_trimmed', 'sum'),
   ).reset_index()

print(gx.round(2).astype(str).to_latex(index=False))

\begin{tabular}{llllll}
\toprule
sample & n_cells & raw_reads & trimmed_reads & raw_Gb & trimmed_Gb \\
\midrule
o1 & 96 & 25488990.0 & 23170059.0 & 33.7 & 26.25 \\
o2 & 92 & 7256000.0 & 639888.0 & 5.32 & 0.68 \\
o3 & 96 & 32878899.0 & 8002384.0 & 32.13 & 14.45 \\
o4 & 96 & 15000216.0 & 6039222.0 & 13.57 & 6.4 \\
\bottomrule
\end{tabular}



# Porechop summaries

In [5]:
dpath = "/scratch/indikar_root/indikar1/shared_data/adapter_clean_up/reports/porechop_summary/"

def parse_adapter_stats(file_path):
    """
    Reads adapter trimming stats and returns a dictionary.
    """
    results = {}
    with open(file_path, 'r') as file:
        for i, line in enumerate(file):
            numbers = re.findall(r'\d[\d,]*', line)
            numbers = [int(x.replace(",", "")) for x in numbers]
            
            if i == 0:
                results['total_reads'] = numbers[1]
                results['start_reads_trimmed'] = numbers[0]
                results['start_bp_trimmed'] = numbers[2]
            elif i == 1:
                results['end_reads_trimmed'] = numbers[0]
                results['end_bp_trimmed'] = numbers[2]
            elif i == 2:
                results['middle_reads_split'] = numbers[0]
            else:
                continue

    return results

res = []

for f in os.listdir(dpath):
    sample = f[:2]
    cell = f[2:5]
    
    fpath = f"{dpath}{f}"
    row = parse_adapter_stats(fpath)
    row['sample'] = sample
    row['cell'] = cell    
    res.append(row)


res = pd.DataFrame(res)
print(res.shape)
outpath = "/scratch/indikar_root/indikar1/shared_data/adapter_clean_up/reports/porechop_summary_aggregated.csv"
res.to_csv(outpath, index=False)
res.head()

(380, 8)


,total_reads,start_reads_trimmed,start_bp_trimmed,end_reads_trimmed,end_bp_trimmed,middle_reads_split,sample,cell
0,16000,15995,1849179,15934,1455635,2350,o2,b18
1,76608,76447,9058848,72620,5412440,43718,o3,b29
2,132238,131808,15408140,130512,11831657,15224,o3,b08
3,30757,30688,3630436,29580,2273752,12964,o3,b14
4,233049,232437,27230332,230220,20396892,25595,o4,b84


In [6]:
clean

,file,num_seqs,Gb,avg_len,N50
0,o4b01.raw.fastq,16284.0,0.000593,36.4,69.0
1,o4b02.raw.fastq,3314.0,0.000282,85.2,583.0
2,o4b03.raw.fastq,785.0,0.000052,66.6,1106.0
3,o4b04.raw.fastq,22499.0,0.018929,841.3,1502.0
4,o4b05.raw.fastq,20530.0,0.002451,119.4,639.0
...,...,...,...,...,...
375,o3b14.raw.fastq,17273.0,0.018743,1085.1,1196.0
376,o1b73.raw.fastq,4798.0,0.005047,1051.8,1202.0
377,o1b47.raw.fastq,490183.0,0.547869,1117.7,1328.0
378,o1b25.raw.fastq,485925.0,0.766367,1577.1,2090.0


In [7]:
clean['cell'] = clean['file'].str[2:5]
clean['sample'] = clean['file'].str[:2]

gx = clean.groupby('sample').agg(
       n_cells = ('file', 'count'),
       total_reads = ('num_seqs', 'sum'),
       reads_per_cell = ('num_seqs', 'mean'),
       average_N50 = ('N50', 'mean'),
   ).reset_index()

print(gx.round(2).astype(str).to_latex(index=False))

\begin{tabular}{lllll}
\toprule
sample & n_cells & total_reads & reads_per_cell & average_N50 \\
\midrule
o1 & 96 & 23170059.0 & 241354.78 & 1374.91 \\
o2 & 92 & 639888.0 & 6955.3 & 1281.53 \\
o3 & 96 & 8002384.0 & 83358.17 & 1800.68 \\
o4 & 96 & 6039222.0 & 62908.56 & 954.75 \\
\bottomrule
\end{tabular}

